In [ ]:
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from azureml.core.workspace import Workspace
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

In [ ]:
datastore_path = "https://raw.githubusercontent.com/thomascjw30/ML-Engineer-Capstone-Project/main/WA_Fn-UseC_-Telco-Customer-Churn.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)
df =ds.to_pandas_dataframe()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df = ds.to_pandas_dataframe().dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.head(5)

In [ ]:
df.MultipleLines.unique()